In [1]:
import os

In [2]:
import tensorflow as tf
tf.config.run_functions_eagerly(True)
tf.compat.v1.enable_eager_execution()

In [3]:
os.chdir("../")
print(os.getcwd())

c:\Users\jv3250328\Downloads\End_To_End_Machine_Learning_Project_MLOps


In [4]:
from pathlib import Path
from dataclasses import dataclass

@dataclass(frozen = True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    param_epochs: int
    param_batch_size: int
    param_is_augmentation: bool
    param_image_size: list

@dataclass(frozen = True)    
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path    

In [5]:
import tensorflow as tf
from End_to_End_ML_Pipeline.constants import *
from End_to_End_ML_Pipeline.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
        
    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])
        
        prepare_callback_config = PrepareCallbacksConfig(
            root_dir = Path(config.root_dir),
            tensorboard_root_log_dir = Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath = Path(config.checkpoint_model_filepath)
        )
        
        return prepare_callback_config
    
    
    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(
            self.config.data_ingestion.unzip_dir,
            "Chicken-fecal-images"
        )
        
        create_directories([
            Path(training.root_dir)
        ])
        
        training_config = TrainingConfig(
            root_dir = Path(training.root_dir),
            trained_model_path = Path(training.trained_model_path),
            updated_base_model_path = Path(prepare_base_model.updated_base_model_path),
            training_data = Path(training_data),
            param_epochs = params.EPOCHS,
            param_batch_size = params.BATCH_SIZE,
            param_is_augmentation = params.AUGMENTATION,
            param_image_size = params.IMAGE_SIZE
        )
        
        return training_config    

In [7]:
import time

In [8]:
class PrepareCallbacks:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config
    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_runnning_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}"
        )
        return tf.keras.callbacks.TensorBoard(log_dir = tb_runnning_log_dir)
    
    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath = self.config.checkpoint_model_filepath,
            save_best_only = True
        )
        
    def get_tb_ckpt_callbacks(self):
        return [self._create_tb_callbacks, self._create_ckpt_callbacks]  

In [9]:
import os
import time
import tensorflow as tf
from zipfile import ZipFile
import urllib.request as request

In [10]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
        
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )
    
    def recompile_model(self):
        # Always use a new optimizer instance when compiling the model
        self.model.compile(
            optimizer = tf.keras.optimizers.Adam(),
            loss = tf.keras.losses.CategoricalCrossentropy(),
            metrics = ["accuracy"]
        )
    
    def train_valid_generator(self):
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.20
        )
        
        dataflow_kwargs = dict(
            target_size = self.config.param_image_size[:-1],
            batch_size = self.config.param_batch_size,
            interpolation = "bilinear"
        )
        
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )
        
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory = self.config.training_data,
            subset = "validation",
            shuffle = False,
            **dataflow_kwargs
        )
        
        if self.config.param_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range = 40,
                horizontal_flip = True,
                width_shift_range = 0.2,
                height_shift_range = 0.2,
                shear_range = 0.2,
                zoom_range = 0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator
            
        self.train_generator = train_datagenerator.flow_from_directory(
            directory = self.config.training_data,
            subset = "training",
            shuffle = True,
            **dataflow_kwargs
        )
        
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)
        
    def train(self, callback_list: list):
        # Recompile the model with a new optimizer instance before training
        self.recompile_model()
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size
        
        self.model.fit(
            self.train_generator,
            epochs = self.config.param_epochs,
            steps_per_epoch = self.steps_per_epoch,
            validation_steps = self.validation_steps,
            validation_data = self.valid_generator,
            callbacks = callback_list
        )
        
        self.save_model(
            path = self.config.trained_model_path,
            model = self.model
        )

In [11]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallbacks(config = prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()
    
    training_config = config.get_training_config()
    training = Training(config = training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train(callback_list = callback_list)
except Exception as e:
    raise e

[2025-08-24 17:25:27,944: INFO: common]: yaml file: config\config.yaml loaded successfully
[2025-08-24 17:25:27,948: INFO: common]: yaml file: params.yaml loaded successfully
[2025-08-24 17:25:27,952: INFO: common]: Directory created at: artifacts
[2025-08-24 17:25:27,955: INFO: common]: Directory created at: artifacts\prepare_callbacks\checkpoint
[2025-08-24 17:25:27,958: INFO: common]: Directory created at: artifacts\prepare_callbacks\tensorboard_log_dir
[2025-08-24 17:25:27,948: INFO: common]: yaml file: params.yaml loaded successfully
[2025-08-24 17:25:27,952: INFO: common]: Directory created at: artifacts
[2025-08-24 17:25:27,955: INFO: common]: Directory created at: artifacts\prepare_callbacks\checkpoint
[2025-08-24 17:25:27,958: INFO: common]: Directory created at: artifacts\prepare_callbacks\tensorboard_log_dir
[2025-08-24 17:25:27,960: INFO: common]: Directory created at: artifacts\training
[2025-08-24 17:25:27,960: INFO: common]: Directory created at: artifacts\training
[2025

c:\Users\jv3250328\.conda\envs\chicken\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
c:\Users\jv3250328\.conda\envs\chicken\Lib\site-packages\tensorflow\python\data\ops\structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(
c:\Users\jv3250328\.conda\envs\chicken\Lib\site-packages\tensorflow\python\data\ops\structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data

19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 8s/step - accuracy: 0.5019 - loss: 10.5197

c:\Users\jv3250328\.conda\envs\chicken\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


[2025-08-24 17:28:13,443: WARNING: saving_api]: You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. 
19/19 ━━━━━━━━━━━━━━━━━━━━ 163s 9s/step - accuracy: 0.4764 - loss: 5.7318 - val_accuracy: 0.6094 - val_loss: 0.6873
19/19 ━━━━━━━━━━━━━━━━━━━━ 163s 9s/step - accuracy: 0.4764 - loss: 5.7318 - val_accuracy: 0.6094 - val_loss: 0.6873
[2025-08-24 17:28:13,997: WARNING: saving_api]: You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. 
[2025-08-24 17:28:13,997: WARNING: saving_api]: You are saving your model as an HDF5 file via `model.save()` o